# Sensor Data Analysis for recognizing letters

Required packages for this notebook:

```
pip install matplotlib numpy ipympl ipywidgets
```

In [ ]:
import dataclasses
import pathlib
from typing import List, Tuple

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

%matplotlib widget

@dataclasses.dataclass
class Recording:
  id: str
  gravity: np.ndarray
  quat: np.ndarray

def loadData() -> List[Recording]:
  result = []
  root = pathlib.Path("feb22")
  for dir in root.glob("M_*"):
    gravity = loadNumeric(dir / "gravity_data.txt")
    quat = loadNumeric(dir / "quat_data.txt")
    result.append(Recording(id=dir.name, gravity=gravity, quat=quat))
  return result

def loadNumeric(path: pathlib.Path) -> np.ndarray:
  rows = []
  for line in open(path, 'r'):
    if not line.strip():
      continue#
    rows.append(np.array([float(v) for v in line.split()], np.float64))
  return np.stack(rows, axis=0)

recordings = loadData()


In [ ]:
# Validate that gravity vectors are about 9.81 m/s^2
def gravityNormsHistogram(recordings):
  gravity_norms = np.concatenate(
    [np.linalg.norm(recording.gravity, axis=1) for recording in recordings])
  plt.hist(x=gravity_norms, bins=30)
  plt.xlabel("Gravity vector magnitude")
  plt.ylabel("Datapoint count")
  return plt
gravityNormsHistogram(recordings).show()

In [ ]:
# John says each gravity list is missing the first 10 elements to align with quaternions. Verify:
def validateDimensions(recordings):
  for recording in recordings:
    g_dim = recording.gravity.shape[0]
    quat_dim = recording.quat.shape[0]
    if g_dim + 10 != quat_dim:
      raise ValueError(f"Recording {recording.id} has {g_dim} gravity readings and {quat_dim} quat entries")
validateDimensions(recordings)

In [ ]:
# What's the average gravity vector?
def meanGravity(recordings, firstn=None):
  if firstn:
    return np.mean(np.concatenate(
        [recording.gravity[:firstn, :] for recording in recordings], axis=0), axis=0)
  return np.mean(np.concatenate(
    [recording.gravity for recording in recordings], axis=0), axis=0)

# Results show that the gravity vectors don't tend to be consistent between episodes.
# This could be due to mounting differences, differences in how the wand is held, or calibration.
print("Overall mean:", meanGravity(recordings), np.linalg.norm(meanGravity(recordings)))
print("First step mean:", meanGravity(recordings, 1), np.linalg.norm(meanGravity(recordings, 1)))

In [ ]:
# Find a relation between the quaternions and the gravity vector.

def quatsToRots(quats) -> np.ndarray:
  """Given a stack of quats, returns a stack of rotation matrices."""
  q0s = quats[:, 0]
  q1s = quats[:, 1]
  q2s = quats[:, 2]
  q3s = quats[:, 3]

  r00 = 2 * (q0s * q0s + q1s * q1s) - 1
  r01 = 2 * (q1s * q2s - q0s * q3s)
  r02 = 2 * (q1s * q3s + q0s * q2s)
  r0 = np.stack([r00, r01, r02], axis=-1)
  
  r10 = 2 * (q1s * q2s + q0s * q3s)
  r11 = 2 * (q0s * q0s + q2s * q2s) - 1
  r12 = 2 * (q2s * q3s - q0s * q1s)
  r1 = np.stack([r10, r11, r12], axis=-1)

  r20 = 2 * (q1s * q3s - q0s * q2s)
  r21 = 2 * (q2s * q3s + q0s * q1s)
  r22 = 2 * (q0s * q0s + q3s * q3s) - 1
  r2 = np.stack([r20, r21, r22], axis=-1)

  return np.stack([r0, r1, r2], axis=1)

def rotatedGravity(recording: Recording, transpose: bool = False):
  R = quatsToRots(recording.quat[10:])
  if transpose:
    R = np.transpose(R, (0, 2, 1))
  return np.matmul(R, np.expand_dims(recording.gravity, 2))

def meanRotatedGravity(recordings: List[Recording]) -> np.ndarray:
  return np.mean(np.concatenate([rotatedGravity(r) for r in recordings], axis=0), axis=0)

# Prints a vector like
# [[ 0.23779003]
#  [-0.37204996]
#  [ 9.7412319 ]]
# which suggests that:
# 1. This is the correct transformation (close to 9.81 in the same direction)
# 2. The base orientation has Z going downwards
print(meanRotatedGravity(recordings),
      np.linalg.norm(meanRotatedGravity(recordings)))


So now we have a way of taking a vector in the wand frame (e.g. gravity)
and getting a result in the world frame, but the world frame has positive Z going downwards.
We should be able to pick a point along the wand line (e.g. [1, 0, 0]), transform it with the
quaternions, and get its trajectory in the world frame.

In [ ]:
def get3DTipTrajectory(recording: Recording, tip=None):
  if tip is None:
    # Direction of the wand tip relative to quat frame.
    # Chosen by trial and error to give positive Z values and clear shapes
    tip = np.array([[1, 0, 0]], dtype=np.float64).T
    tip /= np.linalg.norm(tip)

  # Drop first 10 entries as gravity is not calibrated
  quats = recording.quat[10:]

  R = quatsToRots(quats)
  return np.squeeze(np.matmul(R, tip), -1)

def plot3DTrajectory(ax, trajectory):
  [l.remove() for l in ax.lines]
  xs = trajectory[:, 0]
  ys = trajectory[:, 1]
  zs = trajectory[:, 2]
  ax.scatter(xs, ys, zs, marker='o')
  ax.scatter([0], [0], [0], marker='x')
  ax.set_xlabel('X')
  ax.set_ylabel('Y')
  ax.set_zlabel('Z')
  ax.set_xbound(-1, 1)
  ax.set_ybound(-1, 1)
  ax.set_zbound(-1, 1)

traj_fig = plt.figure('All Trajectories', clear=True)
traj_fig.clear()
traj_ax = traj_fig.add_subplot(projection='3d')

for recording in recordings:
  plot3DTrajectory(traj_ax, get3DTipTrajectory(recording))


The previous cell shows that largely, we have decent trajectories, aligned with positive Z being upwards.

Now we want to project each trajectory into a small 2D image. The idea is to convert the trajectory back to Euler angles, find the smallest yaw and pitch ranges that encompass the whole trajectory, and treat those values as X and Y in the image space.

In [ ]:
# Find the smallest angle range around Z that encompasses a trajectory
def yawAngles(trajectory):
  return np.arctan2(trajectory[:, 0], trajectory[:, 1])

def pitchAngles(trajectory):
  return np.arcsin(trajectory[:, 2])

def wrappingAngleRange(angles) -> Tuple[float, float]:
  angles = np.sort(angles)
  # find largest gap between adjacent angles
  angles_diff = angles - np.roll(angles, 1)
  angles_diff[0] = 2*np.pi + angles_diff[0]
  first_angle = np.argmax(angles_diff)
  last_angle = (first_angle - 1) % angles.size
  return (angles[first_angle], angles[last_angle])

for i in range(len(recordings)):
  angle_range = wrappingAngleRange(yawAngles(get3DTipTrajectory(recordings[i])))
  print(i, "[%0.1f, %0.1f]" % (angle_range[0] * 180 / np.pi, angle_range[1] * 180 / np.pi))


In [ ]:
def normaliseTrajectory(trajectory):
  """Returns trajectory points as yaw and pitch angles, normalised to start at 0."""
  yaw = yawAngles(trajectory)
  pitch = pitchAngles(trajectory)
  yaw_range = wrappingAngleRange(yaw)
  pitch_range = wrappingAngleRange(pitch)
  yaw -= yaw_range[0]
  yaw[yaw < 0] += 2 * np.pi
  pitch = pitch - pitch_range[0]
  pitch[pitch < 0] += 2 * np.pi
  return yaw, pitch

def plotNormalized(ax, trajectory, name):
  yaw, pitch = normaliseTrajectory(trajectory)
  ax.scatter(x=yaw, y=pitch, alpha=0.1, label=name)

normalized_fig = plt.figure("norm")
normalized_fig.clear()
normalized_ax = normalized_fig.add_subplot()
for i, recording in enumerate(recordings):
  plotNormalized(normalized_ax, get3DTipTrajectory(recording), str(i))
normalized_ax.set_xbound(0, 1.5)
normalized_ax.set_ybound(0, 1.5)
normalized_ax.axis('equal')
normalized_ax.legend()